In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob
import os
import polars as pl
# import duckdb as dd
from tqdm import tqdm
import matplotlib.pyplot as plt
import cv2
from pydicom import dcmread
import warnings
from sklearn.preprocessing import LabelEncoder
import pickle
import gc
import ctypes
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
"""from tensorflow.python.keras import backend as K
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))
K.set_session(sess)"""

2024-08-16 08:27:33.090090: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-16 08:27:33.090240: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-16 08:27:33.277174: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


'from tensorflow.python.keras import backend as K\nsess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))\nK.set_session(sess)'

In [2]:
"""import tensorflow as tf
try: # detect TPUs
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
except ValueError: # detect GPUs
    strategy = tf.distribute.MirroredStrategy() # for GPU or multi-GPU machines
    #strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
    #strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy() # for clusters of multi-GPU machines

print("Number of accelerators: ", strategy.num_replicas_in_sync)"""

Number of accelerators:  1


In [3]:
model = keras.models.\
load_model("/kaggle/input/keras_base_lnfn_l1_l2/keras/default/1/keras_base_left_neural_foraminal_narrowing_l1_l2.h5")

In [4]:
submission = pd.read_csv('/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/sample_submission.csv')
submission['row_id'] = 'samples'
#submission

In [5]:
Test = True
config = {}

if Test:
    config['root_file_path'] = '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images'
    config['start'] = 10
    config['end'] = 110
    studies = os.listdir(config['root_file_path'])[config['start']:config['end']]
    test_dict = {}
    image_files = []

    for study in studies:
        for dirname, _, filenames in os.walk(config['root_file_path']+'/'+study):
            for filename in filenames:
                test_dict[os.path.join(dirname, filename).split('/')[-3]] = image_files
                image_files.append(os.path.join(dirname, filename))
else:
    config['root_file_path'] = '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images/'
    test_dict = {}
    image_files = []

    for dirname, _, filenames in os.walk(config['root_file_path']):
        for filename in filenames:
            test_dict[os.path.join(dirname, filename).split('/')[-3]] = image_files
            image_files.append(os.path.join(dirname, filename))

In [20]:
def get_feature_array(img_file_list):
    #pbar.update(1)
    dicom_ds = dcmread(img_file_list)
    img_array = dicom_ds.pixel_array
    return np.resize(np.mean(img_array.T, axis=0),(128,))

vfunc = np.vectorize(get_feature_array, otypes=[object])

In [16]:
def get_predictions(key):
    pbar.update(1)
    final_feature_list = vfunc(test_dict[key]).tolist()
    final = np.array(final_feature_list)
    return model.predict(final)

vfunc2 = np.vectorize(get_predictions, otypes=[object])

#### Snippet for test run only 

In [18]:
list(test_dict.keys())

['2532413137',
 '4031357862',
 '3731783147',
 '4115208111',
 '3588716120',
 '2608235425',
 '1020394063',
 '3563559233',
 '2563313484',
 '82066307',
 '829439070',
 '2864325627',
 '108348787',
 '1153238977',
 '1719776527',
 '2697637622',
 '177339056',
 '3573227658',
 '2925530521',
 '106310815',
 '2493610993',
 '2121891805',
 '1661682358',
 '1768692511',
 '1908143210',
 '247968996',
 '2584853409',
 '2851207875',
 '1782095928',
 '2387323642',
 '2718854859',
 '3367650254',
 '3537214277',
 '1165361924',
 '390498354',
 '3311528906',
 '2155667219',
 '2026420722',
 '3151371929',
 '2605844245',
 '2406108213',
 '2324297179',
 '1722993663',
 '3318343342',
 '2247118504',
 '3426183113',
 '1292979992',
 '2241283141',
 '377474930',
 '4283570761',
 '3889130987',
 '1722308532',
 '2287443993',
 '1094879459',
 '1013589491',
 '3976403280',
 '2988180557',
 '3237383375',
 '2034823515',
 '1057173941',
 '4075603869',
 '1864762516',
 '1934246813',
 '1750792644',
 '2557856398',
 '3192842688',
 '2607713777',
 '15

In [ ]:
rows = {}
global pbar 
pbar = 0

with tqdm(total=len(test_dict.keys())) as pbar:
    y_proba = vfunc2(list(test_dict.keys()))

  0%|          | 0/100 [00:00<?, ?it/s]

238/238 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


  2%|▏         | 2/100 [02:00<1:38:46, 60.48s/it]

238/238 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


  3%|▎         | 3/100 [04:03<2:19:43, 86.43s/it]

238/238 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


  4%|▍         | 4/100 [06:05<2:39:26, 99.65s/it]

In [7]:
# with strategy.scope():
rows = {}
global pbar 
pbar = 0
for st in test_dict.keys():
    with tqdm(total=len(test_dict[st])) as pbar:
        final_feature_list = vfunc(test_dict[st]).tolist()
    final = np.array(final_feature_list)
    y_proba = model.predict(final)
    rows[st] = np.mean(y_proba, axis=0)

100%|██████████| 7586/7586 [02:47<00:00, 45.39it/s]


 42/238 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step

I0000 00:00:1723796487.529223     112 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


238/238 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step


 11%|█         | 815/7586 [00:11<01:34, 71.80it/s] 


KeyboardInterrupt: 

In [7]:
rows = {}
# weight_dict = {'normal_mild':1, 'moderate':2, 'severe':4}
conditions = ['spinal_canal_stenosis', 'neural_foraminal_narrowing', 'subarticular_stenosis']
sides = ['left', 'right']
vertebrae_levels = ['l1_l2', 'l2_l3', 'l3_l4', 'l4_l5', 'l5_s1']
severity_levels = ['normal_mild', 'moderate', 'severe']

for c in conditions:
    for v in vertebrae_levels:
        if c != 'spinal_canal_stenosis':
            for s in sides:
                for st in test_dict.keys():
                    if s+'_'+c+'_'+v == 'left_neural_foraminal_narrowing_l1_l2':
                        #print("going for model ", st+'_'+s+'_'+c+'_'+v)
                        final_feature_list = vfunc(test_dict[st]).tolist()
                        final = np.array(final_feature_list)
                        y_proba = model.predict(final)
                        rows[st+'_'+s+'_'+c+'_'+v] = np.mean(y_proba, axis=0)
                    else:
                        #print(st+'_'+s+'_'+c+'_'+v)
                        rows[st+'_'+s+'_'+c+'_'+v] = np.array([0.333333, 0.333333, 0.333333])
        else:
            for st in test_dict.keys():
                #print(st+'_'+c+'_'+v)
                rows[st+'_'+c+'_'+v] = np.array([0.333333, 0.333333, 0.333333])

KeyboardInterrupt: 

In [10]:
for row_id, feature in tqdm(rows.items()):
    feature_set_reshaped = feature.reshape(1, -1)
    predictions = np.ascontiguousarray(feature_set_reshaped)
    df = pd.DataFrame(predictions, columns=severity_levels)
    df.insert(loc=0, column='row_id', value=row_id)
    submission = pd.concat([submission,df]).reset_index(drop=True)
    
i = submission[(submission.row_id == 'samples')].index
submission = submission.drop(i).reset_index(drop=True)

100%|██████████| 25/25 [00:00<00:00, 755.53it/s]


In [11]:
submission.to_csv('submission.csv', index=False)